In [28]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import requests
import time
from youtube_transcript_api import YouTubeTranscriptApi
import re
import yt_dlp

In [29]:
AUDIO_DIR = "audio"

In [30]:
def get_video_links(driver, channel_url):
    driver.get(channel_url)
    # TODO: stop this from being harcoded
    time.sleep(4)
    video_data = []

    # # Scroll to load videos (modify as necessary to ensure page loads)
    # for _ in range(5):
    #     driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
    #     time.sleep(2)  # adjust as needed

    # Extract the channel name from the channel URL using regex
    pattern = r"youtube\.com\/@([^\/]+)\/videos"
    match = re.search(pattern, channel_url)
    channel_name = match.group(1) if match else "Unknown Channel"
    # print("channel_name: ", channel_name)

    # TODO: fix this
    time.sleep(4)

    # Capture existing video titles without waiting for all to be present
    videos = driver.find_elements(By.ID, 'video-title-link')
    if len(videos) == 0:
        print("shiiiii no vids!")
    
    # Collect data for the first 5 videos
    for vid in videos[:1]:
        print("in the loop!")
        video_title = vid.get_attribute('title')
        link = vid.get_attribute('href')
        # print("video_link: ", link)
        # print("video_title: ", video_title)
        video_data.append({"link": link, "channel": channel_name, "episode": video_title})
    
    return video_data

In [31]:
def get_transcript(id: str) -> None:
    transcript = YouTubeTranscriptApi.get_transcript(
        id, languages=('en', 'en-US', 'en-GB'))
    return transcript

In [32]:
def download_audio(url, file_name):
    print("in download_audio")
    try:
        ydl_opts = {
            'format': 'bestaudio/best',
            'postprocessors': [{
                'key': 'FFmpegExtractAudio',
                'preferredcodec': 'mp3',
                'preferredquality': '192',
            },
            {
                'key': 'FFmpegSplitChapters',  # Adding split chapters processor
                'force_keyframes': True,  # Optional: force keyframes at the start of each chapter
                }],
            'noplaylist': True,
            'writeinfojson': True,  # Optional: write metadata into a JSON file
            'writeannotations': True,  # Optional: write annotations into a file
        }

        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download([url])
    except Exception as e:
        print("Error downloading audio: ", e)

In [33]:
def main():
    driver = webdriver.Chrome()

    channels = [
        "https://www.youtube.com/@a16z/videos",
        "https://www.youtube.com/@Freakonomics/videos",
        # add more channels as needed
    ]

    all_results = []
    for channel in channels:
        new_video_list = get_video_links(driver, channel)
        for video in new_video_list:
            all_results.append(video)

    for video in all_results:
        id = video['link'].split("v=")[1]
        channel = video['channel']
        episode = video['episode']
        video['transcript'] = get_transcript(id)
        video['audio'] = channel + "_" + episode + ".mp3"
        download_audio(video['link'], f"{channel}_{episode}")

    driver.quit()
    return pd.DataFrame(all_results)

# Run the script
df = main()
print(df)

in the loop!
in the loop!
in download_audio
[youtube] Extracting URL: https://www.youtube.com/watch?v=dX7d6bRJI9k
[youtube] dX7d6bRJI9k: Downloading webpage
[youtube] dX7d6bRJI9k: Downloading ios player API JSON
[youtube] dX7d6bRJI9k: Downloading android player API JSON
[youtube] dX7d6bRJI9k: Downloading m3u8 information
[info] dX7d6bRJI9k: Downloading 1 format(s): 251
[info] Writing video metadata as JSON to: Politics & the Future of Tech [dX7d6bRJI9k].info.json


[download] Destination: Politics & the Future of Tech [dX7d6bRJI9k].webm
[download] 100% of  105.43MiB in 00:00:03 at 29.43MiB/s    
[ExtractAudio] Destination: Politics & the Future of Tech [dX7d6bRJI9k].mp3
Deleting original file Politics & the Future of Tech [dX7d6bRJI9k].webm (pass -k to keep)
[SplitChapters] Re-encoding "Politics & the Future of Tech [dX7d6bRJI9k].mp3" with appropriate keyframes
[SplitChapters] Splitting video by chapters; 21 chapters found
[SplitChapters] Chapter 001; Destination: Politics & the Future of Tech - 001 Teaser [dX7d6bRJI9k].mp3
[SplitChapters] Chapter 002; Destination: Politics & the Future of Tech - 002 Intro [dX7d6bRJI9k].mp3
[SplitChapters] Chapter 003; Destination: Politics & the Future of Tech - 003 Why get involved in politics now？ [dX7d6bRJI9k].mp3
[SplitChapters] Chapter 004; Destination: Politics & the Future of Tech - 004 Big Tech's involvement in political process [dX7d6bRJI9k].mp3
[SplitChapters] Chapter 005; Destination: Politics & the F

[download] Destination: 584. How to Pave the Road to Hell ｜ Freakonomics Radio [c3bGpcO_C9k].webm
[download] 100% of   38.04MiB in 00:00:02 at 14.40MiB/s  
[ExtractAudio] Destination: 584. How to Pave the Road to Hell ｜ Freakonomics Radio [c3bGpcO_C9k].mp3
Deleting original file 584. How to Pave the Road to Hell ｜ Freakonomics Radio [c3bGpcO_C9k].webm (pass -k to keep)
[SplitChapters] Chapter information is unavailable
                                          link       channel  \
0  https://www.youtube.com/watch?v=dX7d6bRJI9k          a16z   
1  https://www.youtube.com/watch?v=c3bGpcO_C9k  Freakonomics   

                                             episode  \
0                      Politics & the Future of Tech   
1  584. How to Pave the Road to Hell | Freakonomi...   

                                          transcript  \
0  [{'text': 'you know ignoring Tech uh is kind o...   
1  [{'text': 'hey there it's Steven Dubner before...   

                                              

In [34]:
df.to_csv('ballsack.csv', index=False)